In [85]:
import pandas as pd

# Load your dataset
file_path = "ChatbotData.csv"
df = pd.read_csv(file_path)

# Preview first rows
print(df.head())


             Source                Topic  \
0  Generic Retailer       Order Tracking   
1  Generic Retailer    Returns & Refunds   
2  Generic Retailer  Shipping & Delivery   
3  Generic Retailer  Shipping & Delivery   
4  Generic Retailer    Payment & Billing   

                          Main Question  \
0              How do I track my order?   
1           What is your return policy?   
2          How long does shipping take?   
3  Do you offer international shipping?   
4    What payment methods are accepted?   

                             Alternative Ways to Ask  \
0  Where is my order? | Can I see my delivery sta...   
1   Can I return items? | What's the refund process?   
2  When will my order arrive? | Delivery time est...   
3      Can you ship worldwide? | Ship outside Kenya?   
4  How can I pay? | Which cards/mobile money are ...   

                                   Response Template  \
0  To track your order, **sign into your account ...   
1  We offer a **60-day hass

In [97]:
qa_pairs = {}

for _, row in df.iterrows():
    # Extract the bot’s answer 
    response = row["Simplified Response"]

    # 1. Add the Main Question
    main_q = str(row["Main Question"]).lower().strip()
    qa_pairs[main_q] = response

    # 2. Add Alternative Ways to Ask
    if pd.notna(row["Alternative Ways to Ask"]):
        for alt in row["Alternative Ways to Ask"].split("|"):
            alt_q = alt.lower().strip()
            if alt_q:
                qa_pairs[alt_q] = response

    # 3. Add Simplified Phrasings
    if pd.notna(row["Simplified Phrasings"]):
        for phr in row["Simplified Phrasings"].split("|"):
            phr_q = phr.lower().strip()
            if phr_q:
                qa_pairs[phr_q] = response

print("Total patterns loaded:", len(qa_pairs))


Total patterns loaded: 462


In [147]:
from difflib import get_close_matches

def get_response(user_input):
    user_input = user_input.lower().strip()

    # Step 1: Try exact match
    if user_input in qa_pairs:
        return qa_pairs[user_input]

    # Step 2: Try fuzzy match (similar questions)
    possible_matches = get_close_matches(user_input, qa_pairs.keys(), n=1, cutoff=0.5)

    if possible_matches:
        best_match = possible_matches[0]
        return qa_pairs[best_match]

    # Step 3: Default fallback
    return "Sorry I don't understand that. Could you rephrase your question?"


In [148]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text.strip()


In [157]:
from difflib import SequenceMatcher

def find_best_match(user_q):
    user_q = clean_text(user_q)
    best_score = 0
    best_match = None

    for pattern in qa_pairs.keys():
        score = SequenceMatcher(None, user_q, pattern).ratio()
        if score > best_score:
            best_score = score
            best_match = pattern

    # Only return match if similarity is strong enough
    if best_score > 0.45:
        return best_match
    return None


def get_answer(user_q):
    match = find_best_match(user_q)

    if match:
        return qa_pairs[match]

    return "Sorry I couldn't understand that. Could you ask in a different way?"


In [203]:
import pandas as pd

# Rebuild a simple DataFrame of pattern -> response
patterns = []
for pattern, resp in qa_pairs.items():
    patterns.append([pattern, resp])

patterns_df = pd.DataFrame(patterns, columns=["pattern", "response"])
    

In [204]:
qa_pairs_backup = qa_pairs.copy()     # backup


In [205]:
correct = 0
patterns_df = pd.DataFrame([[pattern, resp] for pattern, resp in qa_pairs.items()], columns=["pattern", "response"])
total = len(patterns_df)

for _, row in patterns_df.iterrows():
    user_q = row["pattern"]
    expected = row["response"]
    predicted = get_answer(user_q)
    if predicted == expected:
        correct += 1

accuracy = correct / total
print(f"Rule-Based Chatbot Accuracy: {accuracy:.2%} ({correct}/{total})")


Rule-Based Chatbot Accuracy: 100.00% (369/369)


In [206]:
# Restore original QA pairs
qa_pairs = qa_pairs_backup


In [207]:
def ask(question):
    print("You:", question)
    print("Bot:", get_answer(question))
    print()
  

In [208]:
# Example interactions
ask("How can I reset my password?")
ask("What is the return policy?")
ask("Tell me about shipping options.")
ask("How do I contact customer support?")
ask("what should i do if i chose pay on delivery (pod)")
ask("how can i contact konga customer service")
ask("does zando offer free returns")
ask("how do i link my safaricom account to masoko")
ask("how do i reset my account password")


You: How can I reset my password?
Bot: Receive **tracking number via email/SMS** to monitor status.

You: What is the return policy?
Bot: **Returns are accepted** within the specified period (e.g., 7-30 days). Check the policy page for details.

You: Tell me about shipping options.
Bot: **Collect your order for free** at a designated secure location.

You: How do I contact customer support?
Bot: Use the **Contact Us page** or **live chat** for support.

You: what should i do if i chose pay on delivery (pod)
Bot: **Do not pay until delivered**. Report early payment requests.

You: how can i contact konga customer service
Bot: **Contact customer service** via the official website's **Email or WhatsApp**.

You: does zando offer free returns
Bot: **Returns are accepted** within the specified period (e.g., 7-30 days). Check the policy page for details.

You: how do i link my safaricom account to masoko
Bot: Account is often **linked to your Safaricom/M-Pesa credentials**.

You: how do i res